In [71]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

generated_guis_path = '../generated_guis'
prompts = ["instruction", "pd_zs", "pd_fs", "ref_instruction"]
metrics = ['nonresponsive', 'overlap', 'boundary', "dimensions", "description", "spacing", "color", "focus", "labels", "errors"]

app_ids = [name for name in os.listdir(generated_guis_path) if os.path.isdir(os.path.join(generated_guis_path, name))]

In [ ]:
df1 = pd.read_csv('./design_evaluation/dimensions_activeness_overlap.csv', delimiter=';')
df2 = pd.read_csv('./description_validation/description_validation.csv', delimiter=';')
df3 = pd.read_csv('./alignment_color_focus.csv', delimiter=',')
df4 = pd.read_csv('./console_errors.csv', delimiter=';')

df_new = pd.DataFrame()
df_new['app_id'] = df1['app_id']
df_new['prompt'] = df1['prompt']
df_new['nonresponsive'] = 1 - (df1['nonresponsive'] / df1['all'])
df_new['overlap'] = 1 - (df1['overlap'] / df1['all'])
df_new['boundary'] = 1 - (df1['boundary'] / df1['all'])
df_new['dimensions'] = 1 - df1['dimensions']
df_new['description'] = df2['correct'] / df2['all']
df_new['spacing'] = df3['spacing_md']
df_new['color'] = df3['color_contrast']
df_new['focus'] = df3['focusable_ratio']
df_new['labels'] = df3['labelled_ratio']
df_new['errors'] = 1 - df4['errors'] / df4['errors'].max()

agg_df = df_new.groupby("app_id")[metrics].mean().reset_index()

summary = agg_df[metrics].describe().T
summary["mean"] = summary["mean"].round(3)
summary["std"] = summary["std"].round(3)
summary["min"] = summary["min"].round(3)
summary["max"] = summary["max"].round(3)
display(summary)

In [ ]:
OUT_DIR = Path("plots")
OUT_DIR.mkdir(exist_ok=True)

metric_vector = df_new.groupby("app_id")[metrics].mean().reset_index()
metric_vector["mean_score"] = metric_vector[metrics].mean(axis=1)

display(metric_vector.sort_values("mean_score", ascending=False).head(10))

for m in metrics:
    plt.figure(figsize=(6,4))
    sns.histplot(df_new[m], bins=20, kde=True)
    plt.title(f"Distribucija metrike: {m}")
    plt.xlabel("Vrednost (0–1)")
    plt.ylabel("Število primerov")
    plt.tight_layout()
    plt.savefig(OUT_DIR / f"dist_{m}.png")
    plt.close()

plt.figure(figsize=(8,5))
sns.boxplot(data=df_new[metrics])
plt.title("Porazdelitev metrik (boxplot)")
plt.ylabel("Vrednost")
plt.tight_layout()
plt.savefig(OUT_DIR / "boxplot_all_metrics.png")
plt.close()

corr = df_new[metrics].corr()
plt.figure(figsize=(6,5))
sns.heatmap(corr, annot=True, cmap="Blues", vmin=0, vmax=1)
plt.title("Korelacijska matrika med metrikami")
plt.tight_layout()
plt.savefig(OUT_DIR / "correlation_heatmap.png")
plt.close()

print("Povzetek korelacij:")
display(corr)

In [ ]:
for m in metrics:
    q1 = df_new[m].quantile(0.25)
    q2 = df_new[m].quantile(0.50)
    q3 = df_new[m].quantile(0.75)
    conditions = [
        df_new[m] < q1,
        (df_new[m] >= q1) & (df_new[m] < q2),
        (df_new[m] >= q2) & (df_new[m] < q3),
        df_new[m] >= q3
    ]
    choices = ["Q1", "Q2", "Q3", "Q4"]
    df_new[f"{m}_quantile"] = np.select(
            conditions,
            choices,
            default=None
        ).astype(object)

mask = pd.DataFrame([
    df_new[f"{m}_quantile"] != "Q1" for m in metrics
]).all(axis=0)

best_app_ids = df_new.loc[mask, "app_id"].unique()

print("Najboljši:", best_app_ids)